# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 17 2022 10:49AM,248737,12,8658580,"SD Head USA, LLC",Released
1,Oct 17 2022 10:46AM,248736,19,TR-RN-10182022,"GCH Granules USA, Inc.",Executing
2,Oct 17 2022 10:43AM,248735,10,SOTAH0000812,"Nextsource Biotechnology, LLC",Released
3,Oct 17 2022 10:42AM,248734,15,PNC397908A,"Person & Covey, Inc.",Released
4,Oct 17 2022 10:42AM,248734,15,PNC397918A,"Person & Covey, Inc.",Released
5,Oct 17 2022 10:42AM,248734,15,PNC397995A,"Person & Covey, Inc.",Released
6,Oct 17 2022 10:42AM,248734,15,PNC397996A,"Person & Covey, Inc.",Released
7,Oct 17 2022 10:42AM,248734,15,PNC397997A,"Person & Covey, Inc.",Released
8,Oct 17 2022 10:42AM,248734,15,PNC397998A,"Person & Covey, Inc.",Released
9,Oct 17 2022 10:42AM,248734,15,PNC397999A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,248732,Released,14
18,248734,Released,9
19,248735,Released,1
20,248736,Executing,1
21,248737,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
248732,NaN,14.0
248734,NaN,9.0
248735,NaN,1.0
248736,1.0,NaN
248737,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248698,4.0,2.0
248702,0.0,6.0
248704,0.0,1.0
248705,0.0,1.0
248706,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248698,4,2
248702,0,6
248704,0,1
248705,0,1
248706,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248698,4,2
1,248702,0,6
2,248704,0,1
3,248705,0,1
4,248706,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248698,4,2
1,248702,,6
2,248704,,1
3,248705,,1
4,248706,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 17 2022 10:49AM,248737,12,"SD Head USA, LLC"
1,Oct 17 2022 10:46AM,248736,19,"GCH Granules USA, Inc."
2,Oct 17 2022 10:43AM,248735,10,"Nextsource Biotechnology, LLC"
3,Oct 17 2022 10:42AM,248734,15,"Person & Covey, Inc."
12,Oct 17 2022 10:24AM,248732,10,Bio-PRF
26,Oct 17 2022 10:19AM,248731,10,ISDIN Corporation
35,Oct 17 2022 10:19AM,248730,10,ISDIN Corporation
86,Oct 17 2022 10:19AM,248726,10,ISDIN Corporation
157,Oct 17 2022 10:16AM,248722,15,"Brookfield Pharmaceuticals, LLC"
178,Oct 17 2022 10:15AM,248723,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 17 2022 10:49AM,248737,12,"SD Head USA, LLC",,1
1,Oct 17 2022 10:46AM,248736,19,"GCH Granules USA, Inc.",1,
2,Oct 17 2022 10:43AM,248735,10,"Nextsource Biotechnology, LLC",,1
3,Oct 17 2022 10:42AM,248734,15,"Person & Covey, Inc.",,9
4,Oct 17 2022 10:24AM,248732,10,Bio-PRF,,14
5,Oct 17 2022 10:19AM,248731,10,ISDIN Corporation,,9
6,Oct 17 2022 10:19AM,248730,10,ISDIN Corporation,,51
7,Oct 17 2022 10:19AM,248726,10,ISDIN Corporation,,71
8,Oct 17 2022 10:16AM,248722,15,"Brookfield Pharmaceuticals, LLC",,21
9,Oct 17 2022 10:15AM,248723,10,ISDIN Corporation,,59


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 10:49AM,248737,12,"SD Head USA, LLC",1,
1,Oct 17 2022 10:46AM,248736,19,"GCH Granules USA, Inc.",,1
2,Oct 17 2022 10:43AM,248735,10,"Nextsource Biotechnology, LLC",1,
3,Oct 17 2022 10:42AM,248734,15,"Person & Covey, Inc.",9,
4,Oct 17 2022 10:24AM,248732,10,Bio-PRF,14,
5,Oct 17 2022 10:19AM,248731,10,ISDIN Corporation,9,
6,Oct 17 2022 10:19AM,248730,10,ISDIN Corporation,51,
7,Oct 17 2022 10:19AM,248726,10,ISDIN Corporation,71,
8,Oct 17 2022 10:16AM,248722,15,"Brookfield Pharmaceuticals, LLC",21,
9,Oct 17 2022 10:15AM,248723,10,ISDIN Corporation,59,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 10:49AM,248737,12,"SD Head USA, LLC",1,
1,Oct 17 2022 10:46AM,248736,19,"GCH Granules USA, Inc.",,1
2,Oct 17 2022 10:43AM,248735,10,"Nextsource Biotechnology, LLC",1,
3,Oct 17 2022 10:42AM,248734,15,"Person & Covey, Inc.",9,
4,Oct 17 2022 10:24AM,248732,10,Bio-PRF,14,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 10:49AM,248737,12,"SD Head USA, LLC",1.0,NaN
1,Oct 17 2022 10:46AM,248736,19,"GCH Granules USA, Inc.",NaN,1.0
2,Oct 17 2022 10:43AM,248735,10,"Nextsource Biotechnology, LLC",1.0,NaN
3,Oct 17 2022 10:42AM,248734,15,"Person & Covey, Inc.",9.0,NaN
4,Oct 17 2022 10:24AM,248732,10,Bio-PRF,14.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 10:49AM,248737,12,"SD Head USA, LLC",1.0,0.0
1,Oct 17 2022 10:46AM,248736,19,"GCH Granules USA, Inc.",0.0,1.0
2,Oct 17 2022 10:43AM,248735,10,"Nextsource Biotechnology, LLC",1.0,0.0
3,Oct 17 2022 10:42AM,248734,15,"Person & Covey, Inc.",9.0,0.0
4,Oct 17 2022 10:24AM,248732,10,Bio-PRF,14.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3233319,281.0,4.0
12,746168,2.0,55.0
15,746175,44.0,0.0
19,248736,0.0,1.0
20,248718,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3233319,281.0,4.0
1,12,746168,2.0,55.0
2,15,746175,44.0,0.0
3,19,248736,0.0,1.0
4,20,248718,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,281.0,4.0
1,12,2.0,55.0
2,15,44.0,0.0
3,19,0.0,1.0
4,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,281.0
1,12,Released,2.0
2,15,Released,44.0
3,19,Released,0.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20
Status,,,,,
Executing,4.0,55.0,0.0,1.0,0.0
Released,281.0,2.0,44.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20
0,Executing,4.0,55.0,0.0,1.0,0.0
1,Released,281.0,2.0,44.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20
0,Executing,4.0,55.0,0.0,1.0,0.0
1,Released,281.0,2.0,44.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()